In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#install reqs
! pip install simpsom
!pip install -q cudf-cu12 cuml-cu12 --extra-index-url=https://pypi.nvidia.com

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 83.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.0/548.0 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.7/849.7 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.4/134.4 kB 110.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.9/196.9 MB 109.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 188.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.2/841.2 kB 183.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.3/513.3 kB 174.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 916.0/916.0 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.5/244.5 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

In [ ]:
import cupy as cp
print(cp.__version__)
!nvcc --version

13.3.0
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import psutil
import simpsom as sps


#butcherbirds = np.load("/content/drive/MyDrive/papagayo/training_data/final_dataset_butcherbird.npy",mmap_mode='r').astype(np.float32)
#superb_lyrebirds = np.load("/content/drive/MyDrive/papagayo/training_data/final_dataset_superb_lyrebird.npy").astype(np.float32)
noisy_friarbirds = np.load("/content/drive/MyDrive/papagayo/training_data/final_dataset_noisy_friarbird.npy").astype(np.float32)
#lorikeets = np.load("/content/drive/MyDrive/papagayo/training_data/final_dataset_lorikeet.npy").astype(np.float32)
#kookaburras = np.load("/content/drive/MyDrive/papagayo/training_data/final_dataset_kookaburra.npy").astype(np.float32)
#magpies = np.load("/content/drive/MyDrive/papagayo/training_data/final_dataset_magpie.npy").astype(np.float32)
#australian_birds = np.load("/content/drive/MyDrive/papagayo/training_data/final_dataset_australian_birds.npy").astype(np.float32)


In [6]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import psutil
import simpsom as sps
import torch
torch.cuda.empty_cache()
#data = np.load("/content/drive/MyDrive/papagayo/training_data/final_dataset.npy").astype(np.float32)
data = noisy_friarbirds
print("Data shape:", data.shape)
print("Data type:", data.dtype)
print("Memory required (GB):", data.nbytes / 1e9)
print("System RAM available (GB):", psutil.virtual_memory().available / 1e9)


scaler = MinMaxScaler()
data = scaler.fit_transform(data)

net = sps.SOMNet(8, 8, data, PBC=True, random_seed=32, GPU=True)
NUM_EPOCHS = 200 # -1 for "until convergence"
net.train(train_algo='batch', start_learning_rate=0.01, epochs=NUM_EPOCHS, batch_size=4096)
net.save(fileName=f"noisy_friarbirds_8x8_som_net_{NUM_EPOCHS}_epochs",out_path="/content/drive/MyDrive/papagayo/som_nets/")

Data shape: (200619, 2049)
Data type: float32
Memory required (GB): 1.644273324
System RAM available (GB): 82.477756416
Periodic Boundary Conditions active.
The weights will be initialized with PCA.
The map will be trained with the batch algorithm.
Training SOM... done!


In [3]:
# For training an upsampling CNN

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

# Load the data from the .npy files
A = np.load("/content/drive/MyDrive/papagayo/upsampling/butcherbird_embeddings.npy").astype(np.float32)  # Shape (N, 8, 8)
B = np.load("/content/drive/MyDrive/papagayo/training_data/final_dataset_butcherbird.npy").astype(np.float32)  # Shape (N, 2049)

print(A.shape, B.shape)
# Convert to torch tensors
X = torch.tensor(A, dtype=torch.float32).unsqueeze(1)  # Adding channel dimension (N, 1, 8, 8)
y = torch.tensor(B, dtype=torch.float32)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Create DataLoader for batching during training
train_data = TensorDataset(X_train, y_train)
val_data = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

# Check if GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define a CNN model
class PredictionCNN(nn.Module):
    def __init__(self):
        super(PredictionCNN, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)  # (1, 8, 8) -> (16, 8, 8)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1) # (16, 8, 8) -> (32, 8, 8)

        # Fully connected layers
        self.fc1 = nn.Linear(32 * 8 * 8, 256)  # 32 channels * 8 * 8 grid
        self.fc2 = nn.Linear(256, 2049)  # Output size is 2049

    def forward(self, x):
        # Apply convolutions with ReLU activations
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))

        # Flatten the output of convolution layers
        x = x.view(x.size(0), -1)  # Flatten (N, 32, 8, 8) -> (N, 32*8*8)

        # Fully connected layers
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)

        return x

# Instantiate the CNN model and move it to the device (GPU or CPU)
model = PredictionCNN().to(device)

# Define the loss function and optimizer
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 200
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)  # Move data to GPU

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = criterion(outputs, targets)

        # Backward pass
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print training progress
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(train_loader)}")

    # Validation phase
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)  # Move data to GPU
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()

        print(f"Validation Loss: {val_loss / len(val_loader)}")

# Save the trained model
torch.save(model.state_dict(), "/content/drive/MyDrive/papagayo/upsampling/prediction_cnn_model_australian_birds.pth")

(661144, 8, 8) (661144, 2049)
Using device: cuda
Epoch [1/200], Loss: 2.251405759573964
Validation Loss: 2.260652361659862
Epoch [2/200], Loss: 2.2501306043226394
Validation Loss: 2.260543645143026
Epoch [3/200], Loss: 2.249995415677006
Validation Loss: 2.260394511290549
Epoch [4/200], Loss: 2.249839825430935
Validation Loss: 2.260288011544976
Epoch [5/200], Loss: 2.2497397402744412
Validation Loss: 2.2598315046164132
Epoch [6/200], Loss: 2.249464232532488
Validation Loss: 2.26097104390157
Epoch [7/200], Loss: 2.249844725399991
Validation Loss: 2.260369177686874
Epoch [8/200], Loss: 2.2498207342618173
Validation Loss: 2.2603708871110944
Epoch [9/200], Loss: 2.2498665499085164
Validation Loss: 2.2603814412758365
Epoch [10/200], Loss: 2.2498502868723325
Validation Loss: 2.2603907295196937
Epoch [11/200], Loss: 2.2498279950651416
Validation Loss: 2.2603522276250296
Epoch [12/200], Loss: 2.2498380201857726
Validation Loss: 2.2603569581974887
Epoch [13/200], Loss: 2.249987215972962
Validati